In [1]:
import pandas as pd
import sys
import os

# Add project root to path
sys.path.append(r"D:\data-problem-solving")

from db import load_table, engine

mis_inventory =  load_table('mis_inventory')

In [2]:
# Step 1: Warehouse config
total_capacity = 500000

# Step 2: item_type_totals
item_type_totals = (
    mis_inventory.groupby("item_type")["square_footage"]
    .sum()
    .reset_index(name="total_square_footage")
)

# Step 3: prime_allocation
prime_total_sqft = item_type_totals.loc[
    item_type_totals["item_type"] == "prime_eligible", "total_square_footage"
].values[0]

prime_sets = total_capacity // prime_total_sqft
prime_space_used = prime_sets * prime_total_sqft
remaining_space = total_capacity - prime_space_used

# Step 4: non_prime_allocation
non_prime_total_sqft = item_type_totals.loc[
    item_type_totals["item_type"] == "not_prime", "total_square_footage"
].values[0]

non_prime_sets = remaining_space // non_prime_total_sqft

# Step 5: item_counts
item_counts = (
    mis_inventory.groupby("item_type")["item_type"]
    .count()
    .reset_index(name="items_per_set")
)

# Step 6: final_results
prime_items_per_set = item_counts.loc[
    item_counts["item_type"] == "prime_eligible", "items_per_set"
].values[0]

non_prime_items_per_set = item_counts.loc[
    item_counts["item_type"] == "not_prime", "items_per_set"
].values[0]

results = pd.DataFrame([
    {"item_type": "prime_eligible", "item_count": prime_sets * prime_items_per_set, "sort_order": 1},
    {"item_type": "not_prime", "item_count": non_prime_sets * non_prime_items_per_set, "sort_order": 2}
])

# Step 7: final output
final_results = results.sort_values("sort_order")["item_type item_count".split()]
print(final_results)


        item_type  item_count
0  prime_eligible      9285.0
1       not_prime         6.0
